In [1]:
import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
%matplotlib inline

ImportError: 
Importing the multiarray numpy extension module failed.  Most
likely you are trying to import a failed build of numpy.
If you're working with a numpy git repo, try `git clean -xdf` (removes all
files not under version control).  Otherwise reinstall numpy.

Original error was: DLL load failed: The specified procedure could not be found.


In [102]:
env = gym.make('FrozenLake-v0')


[2017-06-16 18:10:11,292] Making new env: FrozenLake-v0


In [103]:
tf.reset_default_graph()

In [104]:
#These lines establish the feed-forward part of the network used to choose actions
inputs1 = tf.placeholder(shape=[1,16],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([16,4],0,0.01))
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)

#Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
nextQ = tf.placeholder(shape=[1,4],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

In [108]:
init =tf.global_variables_initializer()

# Set learning parameters
y = .99
e = 0.1
num_episodes = 2000
#create lists to contain total rewards and steps per episode
jList = []
rList = []
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        #Reset environment and get first new observation
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        #The Q-Network
        while j < 99:
            j+=1
            #Choose an action by greedily (with e chance of random action) from the Q-network
            a,allQ = sess.run([predict,Qout],feed_dict={inputs1:np.identity(16)[s:s+1]})
            if np.random.rand(1) < e:
                a[0] = env.action_space.sample()
            #Get new state and reward from environment
            s1,r,d,_ = env.step(a[0])
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Qout,feed_dict={inputs1:np.identity(16)[s1:s1+1]})
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1)
            allQ[0,a[0]] = r + y*maxQ1
            #Train our network using target and predicted Q values
            _,W1=sess.run([updateModel,W],feed_dict={inputs1:np.identity(16)[s:s+1],nextQ:allQ})
            rAll += r
            s = s1
            if d == True:
                #Reduce chance of random action as we train the model.
                e = 1./((i/50) + 10)
                break
        if i%100==1:
            print ("Epoch " + str(i) + " Percent of succesful episodes: " + str(sum(rList)/num_episodes) + "%")        
        jList.append(j)
        rList.append(rAll)
print ("Percent of succesful episodes: " + str(sum(rList)/num_episodes) + "%")

Epoch 1 Percent of succesful episodes: 0.0%
Epoch 101 Percent of succesful episodes: 0.004%
Epoch 201 Percent of succesful episodes: 0.007%
Epoch 301 Percent of succesful episodes: 0.013%
Epoch 401 Percent of succesful episodes: 0.029%
Epoch 501 Percent of succesful episodes: 0.0415%
Epoch 601 Percent of succesful episodes: 0.0655%
Epoch 701 Percent of succesful episodes: 0.088%
Epoch 801 Percent of succesful episodes: 0.1175%
Epoch 901 Percent of succesful episodes: 0.144%
Epoch 1001 Percent of succesful episodes: 0.1755%
Epoch 1101 Percent of succesful episodes: 0.211%
Epoch 1201 Percent of succesful episodes: 0.2405%
Epoch 1301 Percent of succesful episodes: 0.2705%
Epoch 1401 Percent of succesful episodes: 0.299%
Epoch 1501 Percent of succesful episodes: 0.323%
Epoch 1601 Percent of succesful episodes: 0.351%
Epoch 1701 Percent of succesful episodes: 0.383%
Epoch 1801 Percent of succesful episodes: 0.413%
Epoch 1901 Percent of succesful episodes: 0.4425%
Percent of succesful episod

In [ ]:
plt.plot(rList)

class DQN:
    def __init__(self):
        self.discount=.9
        self.epsilon=.1
        self.learning_rate=.03
        self.model=self._build_model()
    def _build_model(self):
        model=Sequential()
        model.add(Dense(64,input_dim=16,activation='linear'))
        model.add(Dense(4,input_dim=16,activation='linear'))
        model.compile(loss="mse",optimizer=SGD(lr=self.learning_rate))
        return model
    def act(self,state):
        a,allQ=self.model.predict(state)
        a=np.argmax(act_values[0])
        if np.random.rand(1) < e:
               a=env.action_space.sample()
        return 

episodes=2000
agent=DQN()
jList = []
rList = []
for i in range(episodes):
    s = env.reset()
    rAll=0
    d=False
    j=0
    while j < 99:
        j+=1
        action=agent.act(s)
        s1,r,d,_=env.step(action)
        
         